# Statistical Analyses
This notebook contains the following information:

*   Fixed Effects Regression Model
  * Statistical analysis
  *   Assumption tests
  * Data visualization
*   Multiple Linear Regression Model
  * Statistical analysis
  * Assumption tests
  * Data visualization


The csv files total_df.csv and total_df_weekly.csv should be present in the same folder as this notebook.



In [ ]:
import pandas as pd
from linearmodels.panel import PanelOLS
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey, linear_harvey_collier, normal_ad
from scipy.stats import shapiro
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from scipy.stats import zscore

In [ ]:
# Load panel data
data = pd.read_csv('total_df.csv')

# Convert 'pcode' to a categorical type and then to numeric codes
data['pcode'] = data['pcode'].astype('category').cat.codes
data['date'] = pd.to_datetime(data['date'])

## Fixed Effects Regression Model

### Statistical analysis

In [ ]:
data.set_index(['pcode', 'date'], inplace=True) # Ensure the data is in long format

# Fixed Effects Model for SS (Self-reported Stress) using physical activity
fe_model_ss_PA = PanelOLS.from_formula('stress ~ screentime + activity_level + EntityEffects', data=data)
fe_results_ss_PA = fe_model_ss_PA.fit()
print(fe_results_ss_PA)

# Fixed Effects Model for SS (Self-reported Stress) using daily calorie expenditure
fe_model_ss_ca = PanelOLS.from_formula('stress ~ screentime + caloriesToday + EntityEffects', data=data)
fe_results_ss_ca = fe_model_ss_ca.fit()
print(fe_results_ss_ca)

# Fixed Effects Model for SS (Self-reported Stress) using activity event
fe_model_ss_AE = PanelOLS.from_formula('stress ~ screentime + active_min + EntityEffects', data=data)
fe_results_ss_AE = fe_model_ss_AE.fit()
print(fe_results_ss_AE)

In [ ]:
# Save results to text files
with open('fe_results_ss_PA.txt', 'w') as f:
    f.write(fe_results_ss_PA.summary.as_text())

with open('fe_results_ss_ca.txt', 'w') as f:
    f.write(fe_results_ss_ca.summary.as_text())

with open('fe_results_ss_AE.txt', 'w') as f:
    f.write(fe_results_ss_AE.summary.as_text())

### Descriptive Statistics

In [ ]:
# Descriptive statistics
summary_stats = data.describe()
print("Summary Statistics:\n", summary_stats)

# Define the color scheme
colors = ['#aee6cb', '#fcddbc']
sns.set_palette(sns.color_palette(colors))

# Plot violin plots for each variable
variables = ['stress', 'screentime', 'caloriesToday']

# Create violin plots
plt.figure(figsize=(15, 5))
for i, var in enumerate(variables):
    plt.subplot(1, len(variables), i + 1)
    sns.violinplot(data[var])
    plt.title(f'Violin plot of {var}')
plt.tight_layout()
plt.savefig('violin_plots.png') # Save violin plots as image files

plt.show()


NameError: name 'data' is not defined

### Assumption tests

The assumptions for a fixed effects model are:


1.   Zero Conditional Mean
2.   Large Outliers are unlikely
3.   No multicolinearity
4.   Cross-sectional dependence



In [ ]:
# 1. Zero Conditional Mean (exogeneity)
sns.residplot(x=fe_results_ss_ca.fitted_values, y=fe_results_ss_ca.resids, lowess=True, line_kws={'color': 'red'})
plt.title('Residuals vs Fitted for fixed effects model')
plt.show()

# Plotting residuals vs. each independent variable
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
data['residuals'] = fe_results_ss_ca.resids
data['screentime']

# residuals against independent variables
ax[0].scatter(data['screentime'], data['residuals'], color="#cdeef3")
ax[0].set_title('Residuals vs screentime')
ax[0].set_xlabel('screentime')
ax[0].set_ylabel('Residuals')
ax[0].axhline(y=0, color='r', linestyle='--')

ax[1].scatter(data['caloriesToday'], data['residuals'], color="#cdeef3")
ax[1].set_title('Residuals vs calories')
ax[1].set_xlabel('calories')
ax[1].set_ylabel('Residuals')
ax[1].axhline(y=0, color='r', linestyle='--')

In [ ]:
# Assumption 2: Large outliers are unlikely

plot_color='#cdeef3'

sns.boxplot(x=fe_results_ss_ca.resids, color='#cdeef3')
plt.title('Boxplot of Residuals for fixed effects model')
plt.show()

# Calculate Z-scores
data['screentime_z'] = zscore(data['screentime'])
outliers_screentime = data[np.abs(data['screentime_z']) > 3] # Identify outliers

# Print the results
if outliers_screentime.empty:
    print("No outliers detected based on screentime Z-scores.")
else:
    print("Outliers based on screentime Z-scores:", outliers_screentime)

# Optional: Plotting for visual confirmation
sns.boxplot(x=data['screentime'], color='#cdeef3')
plt.title('Boxplot of Screentime')
plt.show()

# Calculate Z-scores
data['caloriesToday_z'] = zscore(data['caloriesToday'])
outliers_caloriesToday = data[np.abs(data['caloriesToday_z']) > 3] # Identify outliers

# Print the results
if outliers_caloriesToday.empty:
    print("No outliers detected based on caloriesToday Z-scores.")
else:
    print("Outliers based on caloriesToday Z-scores:", outliers_caloriesToday)

# Optional: Plotting for visual confirmation
sns.boxplot(x=data['caloriesToday'], color='#cdeef3')
plt.title('Boxplot of Calories Today')
plt.show()

In [ ]:
# 3. No Perfect Collinearity (multicolinearity)

# Impute missing values with the mean of the column
data_imputed = data.copy()
data_imputed['screentime'].fillna(data['screentime'].mean(), inplace=True)
data_imputed['caloriesToday'].fillna(data['caloriesToday'].mean(), inplace=True)

# Ensure the columns are not empty after cleaning
if data_imputed.empty:
    raise ValueError("No valid data left after imputing NaNs in 'screentime' and 'caloriesToday'.")

X = data_imputed[['screentime', 'caloriesToday']]
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_data)

In [ ]:
# 4. Cross-sectional dependence

x = fe_results_ss_ca.resids
dw_statistic = sm.stats.durbin_watson(x)
print('Durbin-Watson statistic:', dw_statistic)

### Data visualization

In [ ]:
# Convert 'screentime' from seconds to hours
data['screentime'] = data['screentime'] / 3600

plt.figure(figsize=(20, 6))

# Plot 1: Stress vs. Screen Time with Regression Line
plt.subplot(1, 3, 1)
sns.regplot(x='screentime', y='stress', color='#F45B69', data=data, scatter_kws={'s': 20, 'alpha': 0.8}, line_kws={'color': '#B8DCCC'})
plt.title('Stress Score vs. Screen Time (hours)', fontsize=15, y=1.05)
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='grey', alpha=0.7)
plt.xlabel('Daily Screen Time (hours)', fontsize=12, labelpad=15)
plt.ylabel('Daily Stress Score', fontsize=12, labelpad=15)

# Plot 2: Stress vs. Calories Today with Regression Line
plt.subplot(1, 3, 2)
sns.regplot(x='caloriesToday', y='stress', color='#F45B69', data=data, scatter_kws={'s': 20, 'alpha': 0.8}, line_kws={'color': '#B8DCCC'})
plt.title('Stress Score vs. Calorie Expenditure (kcal)', fontsize=15, y=1.05)
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='grey', alpha=0.7)
plt.xlabel('Daily Calorie Expenditure (kcal)', fontsize=12, labelpad=15)
plt.ylabel('Daily Stress Score', fontsize=12, labelpad=15)

plt.tight_layout()
plt.show()

In [ ]:
# Random sample of 10 participants plotted over time against all variables

# Reload the data for altering
df = pd.read_csv('total_df.csv')

df['date'] = pd.to_datetime(df['date']) # Convert 'date' column to datetime format
df = df.sort_values(by=['pcode', 'date']) # Sort the DataFrame by 'pcode' and 'date'

# Calculate the 'day' number for each date per 'pcode'
df['day'] = df.groupby('pcode')['date'].transform(lambda x: (x - x.min()).dt.days + 1)

df['screentime'] = df['screentime'] / 3600 # Convert 'screentime' from seconds to hours

# Sampling participants
num_samples = 10  # Define number of participants for sampling
unique_pcodes = df['pcode'].unique()
sampled_pcodes = pd.Series(unique_pcodes).sample(n=num_samples, random_state=1)  # random_state for reproducibility
df_sampled = df[df['pcode'].isin(sampled_pcodes)]

# List of variables to plot
variables_to_plot = {
    'stress': 'Stress Score',
    'active_min': 'Activity Event (minutes)',
    'screentime': 'Screen Time (hours)',
    'caloriesToday': 'Calorie Expenditure (kcal)'
}

# Define a pretty palette
palette = sns.color_palette(('#F45B69', '#B8DCCC', '#F5905B', '#BF89D2', '#D1ABAD', '#9FAF90', '#6AD5CB', '#C54451'), n_colors=num_samples)

for var, var_name in variables_to_plot.items():
    plt.figure(figsize=(10, 6))
    ax = sns.lineplot(data=df_sampled, x='day', y=var, hue='pcode', palette=palette)
    ax.set_title(f'{var_name} over Time (days)')  # Set title for the plot
    ax.set_xlabel('Day')  # X-axis label
    ax.set_ylabel(var_name)  # Y-axis label
    # Set grid lines
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='grey', alpha=0.7)
    # Add legend outside the plot
    ax.legend(title='PCode', loc='upper left', bbox_to_anchor=(1, 1))
    # Save the plot as an image file
    plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust layout to make space for the legend
    plt.show()

## Multivariate Regression Model

### Statistical analysis

In [ ]:
# Load weekly data
data_weekly = pd.read_csv('total_df_weekly.csv')

In [ ]:
# Multiple Regression Model for SS
mr_model_ss = sm.OLS.from_formula('stress ~ screentime + caloriesToday', data=data_weekly.reset_index())
mr_results_ss = mr_model_ss.fit()
print(mr_results_ss.summary())

# Multiple Regression Model for DS
mr_model_ds = sm.OLS.from_formula('PHQ ~ screentime + caloriesToday', data=data_weekly.reset_index())
mr_results_ds = mr_model_ds.fit()
print(mr_results_ds.summary())

In [ ]:
# Save results to text files
with open('mr_results_ss.txt', 'w') as f:
    f.write(mr_results_ss.summary().as_text())

with open('mr_results_ds.txt', 'w') as f:
    f.write(mr_results_ds.summary().as_text())

### Descriptive Statistics

In [ ]:
# Descriptive statistics
summary_stats = data_weekly.describe()
print("Summary Statistics:\n", summary_stats)

# plot data distribution depression scores (PHQ)
sns.violinplot(data_weekly['PHQ'])
plt.title(f'Violin plot of depression scores')
plt.tight_layout()
plt.savefig('violin_plot_depression.png')
plt.show()

### Assumption tests

The assumptions for a multiple linear regression model are:


1.   Linearity
2.   Independence
3.   Homoscedasticity
4.   Normality of Residuals
5.   No Multicollinearity


In [ ]:
# Fit the multiple regression models using smf
mr_model_ss = smf.ols('stress ~ screentime + caloriesToday', data=data_weekly).fit()
mr_model_ds = smf.ols('PHQ ~ screentime + caloriesToday', data=data_weekly).fit()

# Define color
plot_color = '#cdeef3'

# Function to check assumptions
def check_assumptions(model, data, dependent_var):
    print(f'\nAssumptions for {dependent_var} model:\n')

    # 1. Linearity
    # Plot residuals vs each predictor
    residuals = model.resid
    predictors = model.model.exog
    predictor_names = model.model.exog_names[1:]

    for predictor in predictor_names:
        plt.figure()
        sns.scatterplot(x=data[predictor], y=residuals, color=plot_color)
        plt.xlabel(predictor)
        plt.ylabel('Residuals')
        plt.title(f'Residuals vs {predictor} ({dependent_var} Model)')
        plt.show()

    # 2. Independence
    # Durbin-Watson test
    dw = sm.stats.durbin_watson(residuals)
    print(f'Durbin-Watson statistic: {dw}')

    # 3. Homoscedasticity
    # Plot residuals vs fitted values
    fitted_values = model.fittedvalues
    plt.figure()
    sns.scatterplot(x=fitted_values, y=residuals, color=plot_color)
    plt.xlabel('Fitted Values')
    plt.ylabel('Residuals')
    plt.title(f'Residuals vs Fitted Values ({dependent_var} Model)')
    plt.show()

    # Breusch-Pagan test
    _, bp_pvalue, _, _ = het_breuschpagan(residuals, predictors)
    print(f'Breusch-Pagan p-value: {bp_pvalue}')

    # 4. Normality of Residuals
    # Q-Q plot
    sm.qqplot(residuals, line='45', color=plot_color)
    plt.title(f'Q-Q Plot ({dependent_var} Model)')
    plt.show()

    # Shapiro-Wilk test
    shapiro_stat, shapiro_pvalue = shapiro(residuals)
    print(f'Shapiro-Wilk test p-value: {shapiro_pvalue}')

    # 5. No Multicollinearity
    # Variance Inflation Factor (VIF)
    vif_data = pd.DataFrame()
    vif_data['Feature'] = predictor_names
    vif_data['VIF'] = [variance_inflation_factor(predictors, i+1) for i in range(len(predictor_names))]
    print(vif_data)

# Check assumptions for both models
check_assumptions(mr_model_ss, data_weekly, 'stress')
check_assumptions(mr_model_ds, data_weekly, 'PHQ')

### Data visualization

In [ ]:
# Convert screentime from seconds to hours
data_weekly['screentime_hours'] = data_weekly['screentime'] / 3600

# Create scatterplot for 'screentime' against 'stress' with regression line
sns.regplot(x='screentime_hours', y='stress', data=data_weekly, scatter_kws={'color': '#F45B69'}, line_kws={'color': '#B8DCCC'})
plt.xlabel('Screen Time (hours)')
plt.ylabel('Stress Score')
plt.title('Scatterplot of Screentime (hours) vs. Stress Score')
plt.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
plt.show()

# Create scatterplot for 'caloriesToday' against 'stress' with regression line
sns.regplot(x='caloriesToday', y='stress', data=data_weekly, scatter_kws={'color': '#F45B69'}, line_kws={'color': '#B8DCCC'})
plt.xlabel('Calorie Expenditure (kcal)')
plt.ylabel('Stress Score')
plt.title('Scatterplot of Calorie Expenditure (kcal) vs. Stress Score')
plt.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
plt.show()

# Create scatterplot for 'screentime' against 'PHQ' with regression line
sns.regplot(x='screentime_hours', y='PHQ', data=data_weekly, scatter_kws={'color': '#F45B69'}, line_kws={'color': '#B8DCCC'})
plt.xlabel('Screentime (hours)')
plt.ylabel('Depression Score (PHQ)')
plt.title('Scatterplot of Screen Time vs. Depression Score (PHQ)')
plt.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
plt.show()

# Create scatterplot for 'caloriesToday' against 'PHQ' with regression line
sns.regplot(x='caloriesToday', y='PHQ', data=data_weekly, scatter_kws={'color': '#F45B69'}, line_kws={'color': '#B8DCCC'})
plt.xlabel('Calorie Expenditure (kcal)')
plt.ylabel('Depression score (PHQ)')
plt.title('Scatterplot of Calorie Expenditure (kcal) vs. Depresion score (PHQ)')
plt.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
plt.show()